# MSADS 507 M4

## Importing Libraries

In [1]:
import numpy as no
import pandas as pd
import pymysql as mysql
import matplotlib.pyplot as plt 
import plotly.express as px
import warnings 
warnings.filterwarnings('ignore')


userName = 'root'
userPass = 'yb%Q8!ec'

## Creating SQL DB Connection

In [2]:
conn=mysql.connect(host='localhost',port=int(3306),user=userName,passwd=userPass,db='gravity_books')

In [3]:
tableNames = pd.read_sql("""SHOW TABLES""", conn) 
tableNames

,Tables_in_gravity_books
0,address
1,address_status
2,author
3,book
4,book_author
5,book_language
6,country
7,cust_order
8,customer
9,customer_address


## Exploratory Data Analysis

In [4]:
# 1. Period: the transaction between ** to **

#latest order date:
a = pd.read_sql_query("""

SELECT order_date 
FROM cust_order
ORDER BY order_date DESC 
LIMIT 1

""" , con = conn) 
a

,order_date
0,2023-02-13 06:37:24


In [5]:
# earliest order date:

b = pd.read_sql_query("""

SELECT order_date 
FROM cust_order 
ORDER BY order_date
LIMIT 1

""" , con = conn) 
b

,order_date
0,2020-02-14 17:22:16


In [6]:
# order counts in the first year:

c = pd.read_sql_query("""

SELECT COUNT(*)
FROM cust_order
WHERE order_date BETWEEN '2020-02-06' AND '2021-02-05'

""" , con = conn) 
c

,COUNT(*)
0,2450


In [7]:
# order counts in the second year:

d = pd.read_sql_query("""

SELECT COUNT(*)
FROM cust_order
WHERE order_date BETWEEN '2021-02-06' AND '2022-02-05'

""" , con = conn) 
d

,COUNT(*)
0,2608


In [8]:
# order counts in the last year:

d = pd.read_sql_query("""

SELECT COUNT(*)
FROM cust_order
WHERE order_date BETWEEN '2022-02-06' AND '2023-02-05'

""" , con = conn) 
d

,COUNT(*)
0,2429


In [9]:
# 2. How many orders and books are sold in total?

e = pd.read_sql_query("""

SELECT COUNT(order_id), COUNT(book_id) books_sold
FROM order_line;

""" , con = conn) 
e

,COUNT(order_id),books_sold
0,15400,15400


In [10]:
# 3. How many customer numbers are in the data?

f = pd.read_sql_query("""

SELECT COUNT(DISTINCT(customer_id)) 
FROM cust_order;

""" , con = conn) 
f

,COUNT(DISTINCT(customer_id))
0,1712


In [11]:
# 4. Which customer placed the most orders?

g = pd.read_sql_query("""

SELECT c.customer_id, c.first_name, c.last_name, COUNT(*) 
FROM customer c
LEFT JOIN cust_order co
ON c.customer_id = co.customer_id
GROUP BY c.customer_id
ORDER BY COUNT(DISTINCT( co.order_id)) DESC LIMIT 1;

""" , con = conn) 
g

,customer_id,first_name,last_name,COUNT(*)
0,764,Giuditta,Giorgetti,20


In [12]:
# 5. What shipping method is most preferred, and what percentage of total

h = pd.read_sql_query("""

SELECT sm.method_name, COUNT(*),COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() AS 'Percent of total'
FROM shipping_method sm
RIGHT JOIN cust_order co
ON sm.method_id = co.shipping_method_id
GROUP BY sm.method_name
ORDER BY COUNT(*) DESC;

""" , con = conn) 
h

,method_name,COUNT(*),Percent of total
0,Priority,1902,25.19205
1,Express,1891,25.04636
2,International,1880,24.90066
3,Standard,1877,24.86093


In [13]:
# 6. author who published the most books:

i = pd.read_sql_query("""

SELECT a.author_id, a.author_name, COUNT(DISTINCT(b.book_id)) AS 'Books Published'
FROM author a
INNER JOIN book_author ba
ON a.author_id = ba.author_id
INNER JOIN book b
ON ba.book_id = b.book_id 
GROUP BY a.author_id
ORDER BY COUNT(DISTINCT(b.book_id)) DESC
LIMIT 1;

""" , con = conn) 
i

,author_id,author_name,Books Published
0,8222,Stephen King,70


In [14]:
# 7. best-selling author

j = pd.read_sql_query("""

SELECT a.author_id, a.author_name, COUNT(ol.order_id) AS Num_Sold 
FROM author a
INNER JOIN book_author ba
ON a.author_id = ba.author_id
INNER JOIN book b
ON ba.book_id= b.book_id
INNER JOIN order_line ol
ON b.book_id = ol.book_id
GROUP BY a.author_id
ORDER BY COUNT(ol.order_id) DESC 
LIMIT 1;

""" , con = conn) 
j

,author_id,author_name,Num_Sold
0,8222,Stephen King,97


In [15]:
# 8. best-selling book
# books sold total

k = pd.read_sql_query("""

SELECT COUNT(book_id)
FROM order_line;

""" , con = conn) 
k

,COUNT(book_id)
0,15400


In [16]:
# DISTINCT books sold in total

l = pd.read_sql_query("""

SELECT COUNT(DISTINCT(book_id))
FROM order_line;

""" , con = conn) 
l

,COUNT(DISTINCT(book_id))
0,8311


In [17]:
# best-selling book

m = pd.read_sql_query("""

SELECT b.book_id, b.title, COUNT(*)
FROM book b
RIGHT JOIN order_line ol
ON b.book_id = ol.book_id
GROUP BY b.book_id
ORDER BY COUNT(*) DESC
LIMIT 10;

""" , con = conn) 
m

,book_id,title,COUNT(*)
0,7513,Secret Identity (Lost #2),8
1,1013,Justine (The Alexandria Quartet #1),8
2,3917,Idoru (Bridge #2),7
3,773,Grizzwold,7
4,7451,The Mammoth Book of New Jules Verne Adventures...,7
5,1354,How Green Was My Valley,6
6,2473,Rose in Bloom (Eight Cousins #2),6
7,2386,Shadow & Claw (The Book of the New Sun #1-2),6
8,965,Under the Net,6
9,2810,The Fourth Hand,6


In [18]:
# 9. Which book makes the most profit: 

n = pd.read_sql_query("""

SELECT ol.book_id, b.title, a.author_name, SUM(ol.price) 
FROM order_line ol
LEFT JOIN book b
ON ol.book_id = b.book_id
LEFT JOIN book_author ba
ON b.book_id = ba.author_id
LEFT JOIN author a
ON ba.author_id = a.author_id
GROUP BY ol.book_id 
ORDER BY SUM(ol.price) DESC 
LIMIT 1;

""" , con = conn) 
n

,book_id,title,author_name,SUM(ol.price)
0,5995,Frogs/Assemblywomen/Wealth (Loeb Classical Lib...,Mercedes Lackey,1559.18


In [19]:
# 10. The top five books with the most language versions 

o = pd.read_sql_query("""

SELECT b.title, b.book_id, bl.language_name, COUNT(*) 
FROM book b
LEFT JOIN book_language bl
ON b.language_id = bl.language_id
GROUP BY b.book_id
ORDER BY COUNT(DISTINCT(bl.language_name)) DESC 
LIMIT 5;

""" , con = conn) 
o

,title,book_id,language_name,COUNT(*)
0,The World's First Love: Mary Mother of God,1,United States English,1
1,The Illuminati,2,English,1
2,The Servant Leader,3,English,1
3,What Life Was Like in the Jewel in the Crown: ...,4,English,1
4,Cliffs Notes on Aristophanes' Lysistrata The ...,5,English,1


In [20]:
# 11. Books in different languages rank

p = pd.read_sql_query("""

SELECT bl.language_id, bl.language_name, COUNT(*), COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() AS 'Percent of total'
FROM book_language bl
RIGHT JOIN book b
ON bl.language_id = b.language_id
GROUP BY b.language_id
ORDER BY COUNT(*) DESC;

""" , con = conn) 
p

,language_id,language_name,COUNT(*),Percent of total
0,1,English,8911,80.08448
1,2,United States English,1409,12.66289
2,4,Spanish,218,1.95920
3,5,British English,214,1.92325
4,3,French,144,1.29415
5,10,German,99,0.88973
6,11,Japanese,46,0.41341
7,6,Multiple Languages,19,0.17076
8,14,Chinese,14,0.12582
9,7,Greek,11,0.09886


In [21]:
# 12. Which customer brings in the most profit?

q = pd.read_sql_query("""

SELECT c.customer_id, c.first_name, c.last_name, SUM(ol.price) AS Spent 
FROM customer c
INNER JOIN cust_order co
ON c.customer_id = co.customer_id
INNEr JOIN order_line ol
ON co.order_id = ol.order_id 
GROUP BY c.customer_id
ORDER BY Spent DESC
LIMIT 5;

""" , con = conn) 
q

,customer_id,first_name,last_name,Spent
0,559,Cthrine,Spence,405.99
1,1549,Leda,Futter,405.58
2,1503,Arlen,Plum,383.25
3,1117,Nettie,Halladey,375.91
4,1497,Friedrich,Fallows,365.01


In [22]:
# 13. The top five cities have the most orders placed by customers. 

r = pd.read_sql_query("""

SELECT ad.city, ctr.country_name, COUNT(*), COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() AS 'Percent of total'
FROM address ad
INNER JOIN customer_address ca
ON ad.address_id = ca.address_id
INNER JOIN cust_order co
ON ca.customer_id = co.customer_id
INNER JOIN order_line ol
ON co.order_id = ol.order_id
INNER JOIN country ctr
ON ad.country_id = ctr.country_id
GROUP BY ad.city, ctr.country_name
ORDER BY COUNT(*) DESC 
LIMIT 5;

""" , con = conn) 
r

,city,country_name,COUNT(*),Percent of total
0,Creighton,South Africa,156,0.49047
1,Klau,Indonesia,155,0.48733
2,Xinhua,China,155,0.48733
3,Barbacena,Brazil,152,0.47790
4,Gościeradów,Poland,149,0.46847


## Adding External Data Sources

In [23]:
popdata = pd.read_csv('API_SP.POP.TOTL_DS2_en_csv_v2_4770387_2.csv')
incomedata = pd.read_csv('Metadata_Country_API_SP.POP.TOTL_DS2_en_csv_v2_4770387_2.csv')

In [24]:
print(popdata.shape)
print(popdata.columns)
popdata

(265, 4)
Index(['countrycode', 'countryname', 'population2020', 'population2021'], dtype='object')


,countrycode,countryname,population2020,population2021
0,ABW,Aruba,106585,106537
1,AFE,Africa Eastern and Southern,685112705,702976832
2,AFG,Afghanistan,38972230,40099462
3,AFW,Africa Western and Central,466189102,478185907
4,AGO,Angola,33428486,34503774
...,...,...,...,...
260,XKX,Kosovo,1790133,1786038
261,YEM,"Yemen, Rep.",32284046,32981641
262,ZAF,South Africa,58801927,59392255
263,ZMB,Zambia,18927715,19473125


In [39]:
r = pd.read_sql_query("""
drop table population;
""" , con = conn) 

## May output error table does not exist, this is just helpful for restart.

DatabaseError: Execution failed on sql '
drop table population;
': (1051, "Unknown table 'gravity_books.population'")

In [40]:
r = pd.read_sql_query("""
create TABLE population (
	countrycode varchar(5),
	countryname varchar(100),
	population2020 bigint,
	population2021 bigint
	)
""" , con = conn) 

## Will always output error TypeError: 'NoneType' object is not iterable -- you can ignore

TypeError: 'NoneType' object is not iterable

In [41]:
cursor = conn.cursor()
for i,row in popdata.iterrows():
            sql = "INSERT INTO population VALUES (%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            conn.commit()

In [42]:
pd.read_sql_query("""
select * from population
""" , con = conn)

,countrycode,countryname,population2020,population2021
0,ABW,Aruba,106585,106537
1,AFE,Africa Eastern and Southern,685112705,702976832
2,AFG,Afghanistan,38972230,40099462
3,AFW,Africa Western and Central,466189102,478185907
4,AGO,Angola,33428486,34503774
...,...,...,...,...
260,XKX,Kosovo,1790133,1786038
261,YEM,"Yemen, Rep.",32284046,32981641
262,ZAF,South Africa,58801927,59392255
263,ZMB,Zambia,18927715,19473125


In [ ]:
print(incomedata.shape)
print(incomedata.columns)
incomedata

(265, 4)
Index(['countrycode', 'region', 'incomegroup', 'countryname'], dtype='object')


,countrycode,region,incomegroup,countryname
0,ABW,Latin America & Caribbean,High income,Aruba
1,AFE,NaN,NaN,Africa Eastern and Southern
2,AFG,South Asia,Low income,Afghanistan
3,AFW,NaN,NaN,Africa Western and Central
4,AGO,Sub-Saharan Africa,Lower middle income,Angola
...,...,...,...,...
260,XKX,Europe & Central Asia,Upper middle income,Kosovo
261,YEM,Middle East & North Africa,Low income,"Yemen, Rep."
262,ZAF,Sub-Saharan Africa,Upper middle income,South Africa
263,ZMB,Sub-Saharan Africa,Low income,Zambia


In [ ]:
incomedata = incomedata.dropna()

In [ ]:
r = pd.read_sql_query("""
drop table income;
""" , con = conn) 

## May output error table does not exist, this is just helpful for restart.

TypeError: 'NoneType' object is not iterable

In [ ]:
r = pd.read_sql_query("""
create TABLE income (
	countrycode varchar(5),
	region varchar(255),
	incomegroup varchar(255),
	countryname varchar(100)
)
""" , con = conn) 

## Will always output error TypeError: 'NoneType' object is not iterable -- you can ignore

TypeError: 'NoneType' object is not iterable

In [ ]:
cursor = conn.cursor()
for i,row in incomedata.iterrows():
            sql = "INSERT INTO income VALUES (%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            conn.commit()

In [ ]:
pd.read_sql_query("""
select * from income
""" , con = conn)

,countrycode,region,incomegroup,countryname
0,ABW,Latin America & Caribbean,High income,Aruba
1,AFG,South Asia,Low income,Afghanistan
2,AGO,Sub-Saharan Africa,Lower middle income,Angola
3,ALB,Europe & Central Asia,Upper middle income,Albania
4,AND,Europe & Central Asia,High income,Andorra
...,...,...,...,...
211,XKX,Europe & Central Asia,Upper middle income,Kosovo
212,YEM,Middle East & North Africa,Low income,"Yemen, Rep."
213,ZAF,Sub-Saharan Africa,Upper middle income,South Africa
214,ZMB,Sub-Saharan Africa,Low income,Zambia


In [ ]:
pd.read_sql_query("""
select * from country
""" , con = conn)

,country_id,country_name
0,1,Afghanistan
1,2,Netherlands Antilles
2,3,Albania
3,4,Algeria
4,5,Andorra
...,...,...
227,228,Yugoslavia
228,229,Zambia
229,230,Zimbabwe
230,231,Singapore


In [ ]:
pd.read_sql_query("""
select * from country c
join income i on i.countryname = c.country_name
""" , con = conn)

## There are only matches on 175 rows

,country_id,country_name,countrycode,region,incomegroup,countryname
0,1,Afghanistan,AFG,South Asia,Low income,Afghanistan
1,3,Albania,ALB,Europe & Central Asia,Upper middle income,Albania
2,4,Algeria,DZA,Middle East & North Africa,Lower middle income,Algeria
3,5,Andorra,AND,Europe & Central Asia,High income,Andorra
4,6,Angola,AGO,Sub-Saharan Africa,Lower middle income,Angola
...,...,...,...,...,...,...
170,221,Vietnam,VNM,East Asia & Pacific,Lower middle income,Vietnam
171,229,Zambia,ZMB,Sub-Saharan Africa,Low income,Zambia
172,230,Zimbabwe,ZWE,Sub-Saharan Africa,Lower middle income,Zimbabwe
173,231,Singapore,SGP,East Asia & Pacific,High income,Singapore


In [43]:
pd.read_sql_query("""
select * from country c
join population p on p.countryname = c.country_name
""" , con = conn)

## There are only matches on 175 rows

,country_id,country_name,countrycode,countryname,population2020,population2021
0,11,Aruba,ABW,Aruba,106585,106537
1,1,Afghanistan,AFG,Afghanistan,38972230,40099462
2,6,Angola,AGO,Angola,33428486,34503774
3,3,Albania,ALB,Albania,2837849,2811666
4,5,Andorra,AND,Andorra,77700,79034
...,...,...,...,...,...,...
170,175,Samoa,WSM,Samoa,214929,218764
171,111,Kosovo,XKX,Kosovo,1790133,1786038
172,171,South Africa,ZAF,South Africa,58801927,59392255
173,229,Zambia,ZMB,Zambia,18927715,19473125


## Combining Data and Visualizing

In [72]:
ordersbyincomegroup = pd.read_sql_query("""
select i.incomegroup, count(co.order_id) as totalorders, count(distinct(c.country_name)) as numcountries
from cust_order co
join address a on a.address_id = co.dest_address_id
join country c on a.country_id = c.country_id
join income i on i.countryname = c.country_name
group by i.incomegroup
""" , con = conn)

In [73]:
ordersbyincomegroup

,incomegroup,totalorders,numcountries
0,High income,1393,23
1,Low income,281,9
2,Lower middle income,2120,23
3,Upper middle income,2618,24


In [75]:
fig = px.bar(ordersbyincomegroup, x='incomegroup', y='totalorders',
             title = 'Number of Orders by Country Income Group',
             labels={
                     "incomegroup": "Order Destination Country Income Group",
                     "totalorders": "Total Orders"
            })
fig.show()

In [82]:
import plotly.graph_objects as pg

fig = pg.Figure(data=[pg.Bar(
    name = 'Total Orders',
    x = ordersbyincomegroup.iloc[:, 0],
    y = ordersbyincomegroup.iloc[:, 1]/10 #Divided by 10 to see the country difference beterr
   ),
                       pg.Bar(
    name = 'Total Countries',
    x = ordersbyincomegroup.iloc[:, 0],
    y = ordersbyincomegroup.iloc[:, 2]
   )
])

fig.show()

## We can see that all groups by low income have the same number of countries within them,
# so the differences in total orders can not be explained just by the number of countries within the group

In [46]:
ordersbypop = pd.read_sql_query("""
select c.country_name, p.population2020, count(co.order_id) as totalorders
from cust_order co
join address a on a.address_id = co.dest_address_id
join country c on a.country_id = c.country_id
join population p on p.countryname = c.country_name
group by c.country_name, p.population2020
""" , con = conn)

In [47]:
ordersbypop

,country_name,population2020,totalorders
0,Afghanistan,38972230,101
1,Albania,2837849,30
2,Argentina,45376763,39
3,Armenia,2805608,88
4,Australia,25655289,24
...,...,...,...
74,Uzbekistan,34232050,21
75,Vietnam,96648685,70
76,Samoa,214929,18
77,Kosovo,1790133,2


In [57]:
fig = px.scatter(ordersbypop, x="population2020", y="totalorders",
             title = 'Number of Orders by Country Population',
             labels={
                     "population2020": "Country Population",
                     "totalorders": "Total Orders"
            })
fig.show()

In [49]:
spendbyincomegroup = pd.read_sql_query("""
select i.incomegroup, sum(ol.price) as totalspend
from cust_order co
join order_line ol on ol.order_id = co.order_id
join address a on a.address_id = co.dest_address_id
join country c on a.country_id = c.country_id
join income i on i.countryname = c.country_name
group by i.incomegroup
""" , con = conn)

In [50]:
spendbyincomegroup

,incomegroup,totalspend
0,Low income,5506.93
1,Upper middle income,52630.17
2,High income,28446.51
3,Lower middle income,43895.68


In [59]:
fig = px.bar(spendbyincomegroup, x='incomegroup', y='totalspend',
             title = 'Total Spend by Country Income Group',
             labels={
                     "incomegroup": "Order Destination Country Income Group",
                     "totalspend": "Total Spend"
            })
fig.show()

In [52]:
spendbypop = pd.read_sql_query("""
select c.country_name, p.population2020, sum(ol.price) as totalspend
from cust_order co
join order_line ol on ol.order_id = co.order_id
join address a on a.address_id = co.dest_address_id
join country c on a.country_id = c.country_id
join population p on p.countryname = c.country_name
group by c.country_name, p.population2020
""" , con = conn)

In [53]:
spendbypop

,country_name,population2020,totalspend
0,Afghanistan,38972230,2180.27
1,Albania,2837849,615.52
2,Argentina,45376763,656.30
3,Armenia,2805608,1714.93
4,Australia,25655289,416.09
...,...,...,...
74,Tunisia,12161723,1131.48
75,Uganda,44404611,150.17
76,Ukraine,44132049,1921.40
77,Uzbekistan,34232050,397.00


In [61]:
fig = px.scatter(spendbypop, x="population2020", y="totalspend", text="country_name",
             title = 'Total Spend by Population',
             labels={
                     "population2020": "Country Population",
                     "totalspend": "Total Spend"
            })
fig.show()

In [62]:
spendbypop.loc[spendbypop['country_name'] == 'China']

,country_name,population2020,totalspend
13,China,1411100000,24425.49


In [63]:
spendbypop.loc[spendbypop['country_name'] == 'Indonesia']

,country_name,population2020,totalspend
35,Indonesia,271857970,20560.98


In [64]:
spendbypop = spendbypop.drop([13,35])

In [68]:
fig = px.scatter(spendbypop, x="population2020", y="totalspend", text="country_name", trendline = "ols",
             title = 'Total Spend by Population',
             labels={
                     "population2020": "Country Population",
                     "totalspend": "Total Spend"
            })
fig.show()